In [70]:
import random as rd
import numpy as np


#this just generates a random sequence of bitstream that is whatever length you want

def rand_bitstream(bitlength):
    bitstream = ""
    for i in range(bitlength):
        temp = str(rd.randint(0,1))
        bitstream += temp
    return bitstream



#because I was lazy and didnt want to write this over and over
def con_bin_str(input1):
    step1 = format(input1,'b')
    step2 = str(step1.zfill(96))#hella paranoid
    step3 = step2[:96]#we had cases of output being 97 bits long... can't have that
    return step3

#I wanted to define this so that you wouldn't get a random starting IDP_K data stream Every time 
# you did a protocal run.
IDP_K_str = '111111111000100100001110010001011010011111100010010000000111100101111101110110101101000011000111101101001011100000111010110011100001101110111001111111110100010010010001100000011101000110001001000101101001110010000010111110000011001100001100111101101011000010111100110101100110101000110101110001011010111110101010111100101001111001100101000111010110111101111100100011100010101011000010111110011111110011100000111010011011011100111000110100100110100110011011001011111101011011010110'


In [71]:

def MMAP(oracle):
    
    #this is our ID, never changing, always the same
    ID_MMAP_str ='101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010'
    
    ID_MMAP = int(ID_MMAP_str,2)
    
    IDP_MMAP_str, K_1_MMAP_str, K_2_MMAP_str, K_3_MMAP_str, K_4_MMAP_str = [oracle[i:i+96] for i in range(0,len(oracle),96)]
    
    IDP_MMAP = int(IDP_MMAP_str,2)
    K_1_MMAP = int(K_1_MMAP_str,2)
    K_2_MMAP = int(K_2_MMAP_str,2)
    K_3_MMAP = int(K_3_MMAP_str,2)
    K_4_MMAP = int(K_4_MMAP_str,2)
    
    
    n1_MMAP = int(rand_bitstream(96),2)
    n2_MMAP = int(rand_bitstream(96),2)
    
    A_int_MMAP = IDP_MMAP ^ K_1_MMAP ^ n1_MMAP
    B_int_MMAP = (IDP_MMAP & K_2_MMAP) | n1_MMAP
    C_int_MMAP = IDP_MMAP + K_3_MMAP + n2_MMAP
    D_int_MMAP = (IDP_MMAP | K_4_MMAP) & n2_MMAP
    E_int_MMAP = (IDP_MMAP + ID_MMAP) ^ n1_MMAP
    
    NextIDP_int_MMAP = (IDP_MMAP + (n1_MMAP ^ n2_MMAP)) ^ ID_MMAP
    NextK_1_int_MMAP = K_1_MMAP ^ n1_MMAP ^ (K_3_MMAP + ID_MMAP)
    NextK_2_int_MMAP = K_2_MMAP ^ n2_MMAP ^ (K_4_MMAP + ID_MMAP)
    NextK_3_int_MMAP = (K_3_MMAP ^ n1_MMAP) + (K_1_MMAP ^ ID_MMAP)
    NextK_4_int_MMAP = (K_4_MMAP ^ n1_MMAP) + (K_2_MMAP ^ ID_MMAP)
    
    A_MMAP = con_bin_str(A_int_MMAP)
    B_MMAP = con_bin_str(B_int_MMAP)
    C_MMAP = con_bin_str(C_int_MMAP)
    D_MMAP = con_bin_str(D_int_MMAP)
    E_MMAP = con_bin_str(E_int_MMAP)
    
    NextIDP_MMAP = con_bin_str(NextIDP_int_MMAP)
    NextK_1_MMAP = con_bin_str(NextK_1_int_MMAP)
    NextK_2_MMAP = con_bin_str(NextK_2_int_MMAP)
    NextK_3_MMAP = con_bin_str(NextK_3_int_MMAP)
    NextK_4_MMAP = con_bin_str(NextK_4_int_MMAP)
    
    NextIDP_K_MMAP = NextIDP_MMAP + NextK_1_MMAP + NextK_2_MMAP + NextK_3_MMAP + NextK_4_MMAP
    
    return A_MMAP,B_MMAP,C_MMAP,D_MMAP,E_MMAP,NextIDP_K_MMAP
    
    

    
AMMAP,BMMAP,CMMAP,DMMAP,EMMAP,NextIDP_KMMAP = MMAP(IDP_K_str)








In [72]:
def Fp_func(input1):
    
    chunks = [input1[i:i+4] for i in range(0,len(input1),4)]
    
    final_output_str = ''
    
    for j in range(len(chunks)):
        
        k = 0
        
        while k < len(chunks[j]):
                   
            x1_str, x2_str, x3_str, x4_str = [chunks[j][l:l+1] for l in range(0,len(chunks[j]),1)]
            
            x1 = int(x1_str,2)
            x2 = int(x2_str,2)
            x3 = int(x3_str,2)
            x4 = int(x4_str,2)
            
            output = ((x1 ^ x2) ^ (x3 ^ x4))
            
            output_binary = format(output,'b')
            output_str = str(output_binary)
            
            k = k + 1
            
        final_output_str += output_str
            
    return final_output_str    
   
    
    

In [82]:
def EMAP(oracle):
    
    ID_EMAP_str ='101110011011001011100100001101010011110010001001000101001101001001110010110100100001011001110010'
    
    ID_EMAP = int(ID_EMAP_str,2)
    
    IDP_EMAP_str, K_1_EMAP_str, K_2_EMAP_str, K_3_EMAP_str, K_4_EMAP_str = [oracle[i:i+96] for i in range(0,len(oracle),96)]
    
    IDP_EMAP = int(IDP_EMAP_str,2)
    K_1_EMAP = int(K_1_EMAP_str,2)
    K_2_EMAP = int(K_2_EMAP_str,2)
    K_3_EMAP = int(K_3_EMAP_str,2)
    K_4_EMAP = int(K_4_EMAP_str,2)
    
    
    n1_EMAP = int(rand_bitstream(96),2)
    n2_EMAP = int(rand_bitstream(96),2)
    
    A_int_EMAP = IDP_EMAP ^ K_1_EMAP ^ n1_EMAP
    B_int_EMAP = (IDP_EMAP & K_2_EMAP) | n1_EMAP
    C_int_EMAP = IDP_EMAP + K_3_EMAP + n2_EMAP
    D_int_EMAP = (IDP_EMAP | K_4_EMAP) & n2_EMAP
    E_int_EMAP = (IDP_EMAP + ID_EMAP) ^ n1_EMAP
    
    ID_EMAP_str_1 , ID_EMAP_str_2 = [ID_EMAP_str[i:i+48] for i in range(0,len(ID_EMAP_str),48)]
    
    K_1_Fp_str = Fp_func(K_1_EMAP_str)
    K_2_Fp_str = Fp_func(K_2_EMAP_str)
    K_3_Fp_str = Fp_func(K_3_EMAP_str)
    K_4_Fp_str = Fp_func(K_4_EMAP_str)
    
    Sub_NextK_1_str = ID_EMAP_str_1 + K_4_Fp_str + K_3_Fp_str
    Sub_NextK_2_str = K_1_Fp_str + K_4_Fp_str + ID_EMAP_str_2
    Sub_NextK_3_str = ID_EMAP_str_1 + K_4_Fp_str + K_2_Fp_str
    Sub_NextK_4_str = K_3_Fp_str + K_1_Fp_str + ID_EMAP_str_2
    
    Sub_NextK_1 = int(Sub_NextK_1_str,2)
    Sub_NextK_2 = int(Sub_NextK_2_str,2)
    Sub_NextK_3 = int(Sub_NextK_3_str,2)
    Sub_NextK_4 = int(Sub_NextK_4_str,2)
    
    NextIDP_int_EMAP = IDP_EMAP ^ n2_EMAP ^ K_1_EMAP 
    NextK_1_int_EMAP = K_1_EMAP ^ n2_EMAP ^ Sub_NextK_1
    NextK_2_int_EMAP = K_2_EMAP ^ n2_EMAP ^ Sub_NextK_2
    NextK_3_int_EMAP = K_3_EMAP ^ n1_EMAP ^ Sub_NextK_3
    NextK_4_int_EMAP = K_4_EMAP ^ n1_EMAP ^ Sub_NextK_4
    
    
    A_EMAP = con_bin_str(A_int_EMAP)
    B_EMAP = con_bin_str(B_int_EMAP)
    C_EMAP = con_bin_str(C_int_EMAP)
    D_EMAP = con_bin_str(D_int_EMAP)
    E_EMAP = con_bin_str(E_int_EMAP)
    
    NextIDP_EMAP = con_bin_str(NextIDP_int_EMAP)
    NextK_1_EMAP = con_bin_str(NextK_1_int_EMAP)
    NextK_2_EMAP = con_bin_str(NextK_2_int_EMAP)
    NextK_3_EMAP = con_bin_str(NextK_3_int_EMAP)
    NextK_4_EMAP = con_bin_str(NextK_4_int_EMAP)
    
    NextIDP_K_EMAP = NextIDP_EMAP + NextK_1_EMAP + NextK_2_EMAP + NextK_3_EMAP + NextK_4_EMAP
    
                               
    return A_EMAP, B_EMAP, C_EMAP, D_EMAP, E_EMAP, NextIDP_K_EMAP

In [83]:
EMAP(IDP_K_str)

('111101101110111111101100011110101011110101001000100010000110111001110011101001011111111010101110',
 '101111111101111011011010111100010010001100010011011101110111001110111111111111101111111111100101',
 '100110100111101010011101101111110111101110010000111111100101101111110100001000001100000000001101',
 '101001001011000110101100110001011010011111111000100100100000000011010110000110100000010010000011',
 '100000100111001010010101010001011111001010111100001100010000110000110111101010010000110001101100',
 '111011111000000010001001010011100001101110100111001001001011101100111010010000010000010111100101101010011011101101100011001111101000000011001100011011100001111000101101100110101001100110011011010000111100111001000001001101110100100010011010011110011110010000011000000111100111100011101100110000011100001110010110001101101010001111111111001000001110000010001001111111011101011110000010010001010110111010000001111010010101010101010101111100011110100001110110000000110011111101000100')